#### 先知道每個分類各自的數量

In [1]:
import os

# test is unbalanced dataset

output_folder_authentic = './dataset/PA/PA/ASVspoof2019_PA_train/classification/authentic'
output_folder_fake = './dataset/PA/PA/ASVspoof2019_PA_train/classification/fake'

file_list = os.listdir(output_folder_authentic)
file_count = len(file_list)
file_list2 = os.listdir(output_folder_fake)
file_count2 = len(file_list2)

print(f"The folder train_'{output_folder_authentic[-9:]}' contains {file_count} files.")
print(f"The folder train_'{output_folder_fake[-4:]}' contains {file_count2} files.")

output_folder_devauthentic = './dataset/PA/PA/ASVspoof2019_PA_dev/classification/authentic'
output_folder_devfake = './dataset/PA/PA/ASVspoof2019_PA_dev/classification/fake'

file_list3 = os.listdir(output_folder_devauthentic)
file_count3 = len(file_list3)
file_list4 = os.listdir(output_folder_devfake)
file_count4 = len(file_list4)

print(f"The folder dev_'{output_folder_authentic[-9:]}' contains {file_count3} files.")
print(f"The folder dev_'{output_folder_fake[-4:]}' contains {file_count4} files.")

print(f"Total authentic files: {file_count + file_count3}")
print(f"Total fake files: {file_count2 + file_count4}")

The folder train_'authentic' contains 5400 files.
The folder train_'fake' contains 6695 files.
The folder dev_'authentic' contains 3693 files.
The folder dev_'fake' contains 2881 files.
Total authentic files: 9093
Total fake files: 9576


In [2]:
import os
from PIL import Image
# check picture size

# Define the path to the image
image_path = os.path.join(output_folder_authentic, file_list[0])

# Open the image
image = Image.open(image_path)

# Get the size of the image
image_size = image.size

# Print the size of the image
print(f"The size of the image is: {image_size}")

# Get the mode of the image
image_mode = image.mode

# Extract the channel information from the mode
channels = len(image_mode)

# Print the number of channels
print(f"The image has {channels} channel(s).")

The size of the image is: (1000, 400)
The image has 4 channel(s).


#### 將真跟假包在同一個資料夾中，利用Imagefolder處理

In [3]:
import torchvision.datasets as dset

# 加載圖像數據集，並只考慮指定的資料夾
dataset = dset.ImageFolder(root='./dataset/PA/PA/ASVspoof2019_PA_train/classification')

# 獲取相關信息
print(dataset.classes)  # 根據分的文件夾的名字來確定類別
print(dataset.class_to_idx)  # 按順序為這些類別定義索引為 0, 1, ...
print(dataset.imgs)  # 返回從所有文件夾中獲得的圖片的路徑以及其類別
print(len(dataset))  # 返回數據集的大小

['authentic', 'fake']
{'authentic': 0, 'fake': 1}
[('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000001.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000002.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000003.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000004.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000005.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000006.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000007.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000008.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000009.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classification\\authentic\\PA_T_0000010.png', 0), ('./dataset/PA/PA/ASVspoof2019_PA_train/classific

#### 創造Dataloader

In [17]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset

# 定义数据转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 将图像调整为指定大小
    transforms.ToTensor(),           # 将图像转换为张量
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 归一化图像数据
])

# 创建数据集对象
dataset = dset.ImageFolder(root= './dataset/PA/PA/ASVspoof2019_PA_train/classification', transform=transform)
dev_dataset = dset.ImageFolder(root= './dataset/PA/PA/ASVspoof2019_PA_dev/classification', transform=transform)
# 合并数据集
combined_dataset = ConcatDataset([dataset, dev_dataset])

# 划分数据集
# 首先确定数据集的大小
dataset_size = len(combined_dataset)
# 然后确定划分比例
valid_split = 0.3
# 计算测试集的大小
valid_size = int(valid_split * dataset_size)
# 计算训练集的大小
train_size = dataset_size - valid_size
# 利用 random_split 函数随机划分数据集
train_dataset, valid_dataset = torch.utils.data.random_split(combined_dataset, [train_size, valid_size])

# 创建 DataLoader 对象
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=1024, shuffle=False)
print(len(train_loader))
print(len(valid_loader))

#### Create our own model here

In [5]:
import torch
# True if CUDA is available

# Set device based on whether the cuda is available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


```
class MyModel(nn.Module):
  def __init__(self, in_channels: int=3, img_size: Tuple=(224, 224),
      expand_ratio: int=3, downsamples: int=2, hid_dims: int=32,
      mlp_layers: int=4, num_classes: int=100):
```
- *in_channels: number of **channels** of input image.*
- *img_size: **(height, width)** of input image.*
- *expand_ratio: number of **channels of intermediate convolutional layers**.*
- *downsamples: number of **downsampling convolutional layers**.*
- *hid_dims: number of **features of intermediate linear layers**.*
- *mlp_layers: number of **linear layers**.*
- *num_classes: number of **classification classes**.*

In [6]:
import torch
import torch.nn as nn
from typing import Tuple

class MyModel(nn.Module):
  def __init__(
      self,
      in_channels: int=3,
      img_size: Tuple=(224, 224),
      expand_ratio: int=1,
      downsamples: int=2,
      hid_dims: int=32,
      mlp_layers: int=4,
      num_classes: int=2
      ):
    # Fool-proof mechanism
    assert in_channels > 0, f"Input channels should be greater than 0"
    assert expand_ratio > 0, f"Expand ration should be greater than 0"
    assert mlp_layers > 0, f"Number of mlp layers should be greater than 0"
    assert downsamples > 0, f"Number of downsample layers should be greater than 0"
    assert hid_dims > 0, f"Dims of hidden layers should be greater than 0"
    assert num_classes > 0, f"Number of classes should be greater than 0"

    # Inherit __init__ function from parent class
    super().__init__()

    # Create downsample layers by using several convolutional layers
    hid_channels = int(in_channels * expand_ratio)
    self.downsamples = nn.ModuleList(modules=[
        nn.Conv2d(in_channels=in_channels, out_channels=hid_channels, kernel_size=3, stride=2, padding=1, bias=False),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.BatchNorm2d(num_features=hid_channels), # num_feature indicate input channels
        nn.ReLU6()
    ])
    for downsample in range(downsamples-1):
      self.downsamples.append(nn.Conv2d(in_channels=hid_channels, out_channels=hid_channels, kernel_size=3, stride=2, padding=1, bias=False))
      self.downsamples.append(nn.MaxPool2d(kernel_size=2, stride=2))
      self.downsamples.append(nn.BatchNorm2d(num_features=hid_channels))
      self.downsamples.append(nn.ReLU6())

    # Create mlp layers
    img_h, img_w = img_size
    for _ in range(downsamples):
      if img_h % 2 == 0:
        img_h /= 4
      else:
        img_h = img_h // 4 + 1
    flatten_dims = int(hid_channels * img_h**2)

    self.mlps = nn.ModuleList(modules=[
        nn.Flatten(),
        nn.Linear(in_features=flatten_dims, out_features=int(num_classes if mlp_layers==1 else (flatten_dims:=(flatten_dims//3))), bias=False),
        nn.ReLU6()
    ])
    last_layer = mlp_layers - 1
    for mlp in range(mlp_layers-1):
      if mlp is last_layer-1: # 最後一層
        self.mlps.append(nn.Linear(in_features=flatten_dims, out_features=num_classes, bias=False))
      else:
        self.mlps.append(nn.Linear(in_features=flatten_dims, out_features=(flatten_dims:=(flatten_dims//3)), bias=False))
        self.mlps.append(nn.ReLU6())
        self.mlps.append(nn.Dropout(p=0.5))

  def forward(self, x):
    # Traverse all downsample layers
    for layer in self.downsamples:
      x = layer(x)

    # Traverse all mlp layers
    for layer in self.mlps:
      x = layer(x)

    return x

In [7]:
import torchinfo

# Parameters setting.
batch_size, in_channels, height, width = 256, 4, 224, 224
# Create an input tensor.
input = torch.rand(batch_size, in_channels, height, width)
# Instantiate a model.
model = MyModel(in_channels=in_channels, img_size = (height, width), expand_ratio=2, downsamples=2, hid_dims=32, mlp_layers=4, num_classes=2)
# Forward pass.
output = model(input)
# Print needed info.
print(f"Random input shape is:\n {input.shape}")
print(f"Model output shape is:\n {output.shape}")
print(f"Model's structure:\n {torchinfo.summary(model=model, input_data=input)}")

Random input shape is:
 torch.Size([256, 4, 224, 224])
Model output shape is:
 torch.Size([256, 2])
Model's structure:
Layer (type:depth-idx)                   Output Shape              Param #
MyModel                                  [256, 2]                  --
├─ModuleList: 1-1                        --                        --
│    └─Conv2d: 2-1                       [256, 8, 112, 112]        288
│    └─MaxPool2d: 2-2                    [256, 8, 56, 56]          --
│    └─BatchNorm2d: 2-3                  [256, 8, 56, 56]          16
│    └─ReLU6: 2-4                        [256, 8, 56, 56]          --
│    └─Conv2d: 2-5                       [256, 8, 28, 28]          576
│    └─MaxPool2d: 2-6                    [256, 8, 14, 14]          --
│    └─BatchNorm2d: 2-7                  [256, 8, 14, 14]          16
│    └─ReLU6: 2-8                        [256, 8, 14, 14]          --
├─ModuleList: 1-2                        --                        --
│    └─Flatten: 2-9               

#### Train the model

In [18]:
epochs = 10
lr = 3e-3

In [19]:
import time
# Move model to "device".
model = MyModel().to(device)
# File name.
modelSaveName = "MyModel.pt"
# Create an optimizer.
optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr)
# Create a loss function
lossFunc = nn.CrossEntropyLoss()

bestValidAcc = 0
for epoch in range(1, epochs+1):
  start_time = time.time()  # 計算每個epoch開始的時間
  # Training mode.
  model.train()
  train_loss = 0.0
  for iter, (input, target) in enumerate(train_loader):
    # Move data to "device".
    input = input.to(device)
    target = target.to(device)
    # Forward pass.
    output = model(input)
    # Compute the loss.
    loss=lossFunc(output, target)
    # Clear optimizer gradients.
    optimizer.zero_grad()
    # Loss backward propagation.
    loss.backward()
    # Update all learnable parameters.
    optimizer.step()
    # update training loss
    train_loss += loss.item()*input.size(0)

  if epoch % 5 == 0: 
    # Evaluation mode.
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
      correctCount=0
      for input, target in train_loader:
        input, target = input.to(device), target.to(device)
        output = model(input).max(1)[1]
        correctCount+=torch.sum(output==target).item()
        trainAcc=correctCount/len(train_loader.dataset)

      correctCount=0
      for input, target in valid_loader:
        input, target = input.to(device), target.to(device)
        output = model(input).max(1)[1]
        correctCount+=torch.sum(output==target).item()
        validAcc=correctCount/len(valid_loader.dataset)

      # 計算每個樣本的平均損失
      train_loss = train_loss / len(train_loader.dataset)

    end_time = time.time()  # 計算每個epoch結束的時間
    epoch_time = end_time - start_time  # 計算每個epoch所花的時間
    print("Epoch:{}/{}, Time:{:.2f} seconds, Train acc:{}, Train Loss: {:.4f}, Valid acc:{}".format(epoch, epochs, epoch_time, trainAcc, train_loss, validAcc))
    if validAcc>bestValidAcc:
      bestValidAcc=validAcc
      torch.save(model, modelSaveName)

  else:
    # 計算每個樣本的平均損失
    train_loss = train_loss / len(train_loader.dataset)
    end_time = time.time()  # 計算每個epoch結束的時間
    epoch_time = end_time - start_time  # 計算每個epoch所花的時間
    print("Epoch:{}/{}, Time:{:.2f} seconds, Train Loss: {:.4f}".format(epoch, epochs, epoch_time, train_loss))

print("best valid acc:",bestValidAcc)

Epoch:1/10, Time:27.37 seconds, Train Loss: 0.1632
Epoch:2/10, Time:24.62 seconds, Train Loss: 0.0025
Epoch:3/10, Time:24.70 seconds, Train Loss: 0.0013
Epoch:4/10, Time:24.50 seconds, Train Loss: 0.0004
Epoch:5/10, Time:61.17 seconds, Train acc:1.0, Train Loss: 0.0005, Valid acc:1.0
Epoch:6/10, Time:25.80 seconds, Train Loss: 0.0007
Epoch:7/10, Time:25.00 seconds, Train Loss: 0.0005
Epoch:8/10, Time:25.08 seconds, Train Loss: 0.0002
Epoch:9/10, Time:25.29 seconds, Train Loss: 0.0003
Epoch:10/10, Time:62.65 seconds, Train acc:1.0, Train Loss: 0.0004, Valid acc:1.0
best valid acc: 1.0


#### 納入Test data

In [10]:
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
import torchvision.datasets as dset
# 定义数据转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 将图像调整为指定大小
    transforms.ToTensor(),           # 将图像转换为张量
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 归一化图像数据
])

# 创建数据集对象
test_dataset = dset.ImageFolder(root= './dataset/PA/PA/ASVspoof2019_PA_eval/classification', transform=transform)

# 创建 DataLoader 对象
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

print("test_loader:",len(test_loader))

test_loader: 4


#### Test model

In [20]:
import torch
from sklearn.metrics import f1_score, precision_score, recall_score

# If encounter "AttributeError: 'module' can't find 'MyModel' object", please re-run the cell of "MyModel" class.
model = torch.load("./Mymodel.pt") # load the current cnn model with the highest valid accuracy
model.eval()
with torch.inference_mode():
  correctCount = 0
  predictions = []
  targets = []
  for x, y in test_loader:
    x, y = x.to(device), y.to(device)
    pred = model(x).max(1)[1]
    correctCount += torch.sum(pred == y).item()
    predictions.extend(pred.tolist())
    targets.extend(y.tolist())
  testAcc = correctCount / len(test_loader.dataset)
  print("Testing accuracy of the floating point model: ", testAcc)

f1 = f1_score(targets, predictions)
precision = precision_score(targets, predictions)
recall = recall_score(targets, predictions)

print("F1-score: ", f1)
print("Precision: ", precision)
print("Recall: ", recall)


Testing accuracy of the floating point model:  0.5
F1-score:  0.0
Precision:  0.0
Recall:  0.0


d:\ancode\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
